## Scheduling Real-Time Communication in IEEE 802.1Qbv Time Sensitive Networks

Auther: ilviu S. Craciunas, Ramon Serna Oliver, Martin Chmelík, Wilfried Steiner

Proceedings of the 24th International Conference on Real-Time Networks and Systems

In [18]:
import pandas as pd
import numpy as np
import z3
from tqdm import tqdm


In [19]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

In [20]:
NUM_FLOW = 51
DATA_NAME = "single11"
TOPO_NAME = "0"

task = pd.read_csv("../../data/stream/stream_%s_%s.csv" % (178, 7))
network = pd.read_csv("../../data/stream/stream_topology.csv")

# task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)
# network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## Notice!

Here we assume each traffic is strictly periodic and each traffic only contains 1 frame (Less than 65535)

$f_{i,j + 1}^{(a,b)} = f_{i,j}^{(a,b)} + s_{i}^{(a,b)}.T$

$|F_{i}^{(a,b)}| = \frac{LCM}{s_{i}.T}$

Why?

- To make the comparision more fair as other works only consider 1 frame per Hyper-parameter NUM_FRAME is not common in other works.
- Reasonably, this assumption can be converted to "Each traffic has multiple frames successively or Regard each periodic frame as a unique traffic".
- Reduce complexity, the time for adding constraints is too long.

## 1. Model

In [21]:
s = z3.Solver()
s.set("timeout", time_out * 1000)

Network model:

$\left\langle\left[v_{a}, v_{b}\right] . s,\left[v_{a}, v_{b}\right] . d,\left[v_{a}, v_{b}\right] . m t,\left[v_{a}, v_{b}\right] . c\right\rangle$

In [22]:
net_var = {}
for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['d'] = row['t_proc']
    net_var[row['link']]['c'] = row['q_num']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

Task model：

$\left\langle f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot \phi, f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot T, f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot L\right\rangle$

In [23]:
task_var = {}

In [24]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [25]:
## Assume task is strictly periodic
for i, row in task.iterrows(): 
    task_var.setdefault(i, {})
    route = eval(str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0])))))
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link]['phi'] = z3.Int('phi_' +  str(i) + '_' + str(link))
        task_var[i][link]['p'] =  z3.Int('p_' +  str(i) + '_' + str(link))
        task_var[i][link]['T'] = row['period']
        task_var[i][link]['L'] = int(row['size'] * 8 / net_var[str(link)]['s'])

## 2. Constraints

### 2.1 General constraints

Frame Constraint

$\begin{aligned}
&\forall s_{i} \in \mathcal{S}, \forall\left[v_{a}, v_{b}\right] \in s_{i}, \forall f_{i, j}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{i}^{\left[v_{a}, v_{b}\right]}: \\
&\left(f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot \phi \geq 0\right) \wedge\left(f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot \phi \leq f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot T-f_{i, j}^{\left[v_{a}, v_{b}\right]} \cdot L\right)
\end{aligned}$

In [26]:
for i, f_i in tqdm(task_var.items()):
    for link, f_i_link in f_i.items():
        s.add(f_i_link['phi'] >= 0, f_i_link['phi'] <= f_i_link['T'] - f_i_link['L'])

100%|██████████| 178/178 [00:00<00:00, 1863.80it/s]


In [27]:
s.check()

sat

Flow Transmission Constraint

$\begin{aligned}
&\forall s_{i} \in \mathcal{S}, \forall\left[v_{a}, v_{x}\right],\left[v_{x}, v_{b}\right] \in s_{i} \\
&\forall f_{i, j}^{\left[v_{a}, v_{x}\right]} \in \mathcal{F}_{i}^{\left[v_{a}, v_{x}\right]}, \forall f_{i, j}^{\left[v_{x}, v_{b}\right]} \in \mathcal{F}_{i}^{\left[v_{x}, v_{b}\right]}: \\
&f_{i, j}^{\left[v_{x}, v_{b}\right]} \cdot \phi \times\left[v_{x}, v_{b}\right] . m t-\left[v_{a}, v_{x}\right] . d-\delta \geq \\
&\left(f_{i, j}^{\left[v_{a}, v_{x}\right]} \cdot \phi+f_{i, j}^{\left[v_{a}, v_{x}\right]} . L\right) \times\left[v_{a}, v_{x}\right] . m t
\end{aligned}$

In [28]:
for i in task_var.keys():
    path = list(task_var[i].keys())
    for _i, link in enumerate(path[:-1]):
        next_hop = path[_i + 1]
        s.add(
            task_var[i][link]['phi'] + task_var[i][link]['L'] + net_var[link]['d'] + sync_error <=
            task_var[i][next_hop]['phi']
        )

In [29]:
s.check()

sat

End-to-end constraint

$\begin{aligned}
&\forall s_{i} \in \mathcal{S}: \operatorname{src}\left(s_{i}\right) \cdot m t \times f_{i, 1}^{s r c\left(s_{i}\right)} \cdot \phi+s_{i} \cdot e 2 e \geq \\
&\operatorname{dest}\left(s_{i}\right) \cdot m t \times\left(\operatorname{last}\left(\mathcal{F}_{i}^{\operatorname{dest}\left(s_{i}\right)}\right) \cdot \phi+\operatorname{last}\left(\mathcal{F}_{i}^{\operatorname{dest}\left(s_{i}\right)}\right) \cdot L\right)
\end{aligned}$

In [30]:
for i in tqdm(task_var.keys()):
    _hop_s = list(task_var[i].items())[0]
    _hop_e = list(task_var[i].items())[-1]
    s.add(
        _hop_s[1]['phi'] + int(task.loc[i]['deadline']) >=
        _hop_e[1]['phi'] + _hop_e[1]['L'] + net_var[_hop_e[0]]['d'] + sync_error
        # _hop_e[1]['phi'] + _hop_e[1]['L'] + sync_error
    )

100%|██████████| 178/178 [00:00<00:00, 1942.39it/s]


In [31]:
s.check()

sat

Link Constraint

$\begin{aligned}
&\forall\left[v_{a}, v_{b}\right] \in \mathcal{L}, \forall \mathcal{F}_{i}^{\left[v_{a}, v_{b}\right]}, \mathcal{F}_{j}^{\left[v_{a}, v_{b}\right]}, i \neq j \\
&\forall f_{i, k}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{i}^{\left[v_{a}, v_{b}\right]}, \forall f_{j, l}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{j}^{\left[v_{a}, v_{b}\right]} \\
&\forall \alpha \in\left[0, h p_{i}^{j} / s_{i} \cdot T-1\right], \forall \beta \in\left[0, h p_{i}^{j} / s_{j} \cdot T-1\right]: \\
&\left(f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot \phi+\alpha \times f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot T \geq\right. \\
&\left.f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot \phi+\beta \times f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot T+f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot L\right) \vee \\
&\left(f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot \phi+\beta \times f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot T \geq\right. \\
&\left.f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot \phi+\alpha \times f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot T+f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot L\right)
\end{aligned}$

In [32]:
count = 0
for link in tqdm(net_var.keys()):
    for i, j in [(i,j) for i in range(len(task)) for j in range(0, len(task)) 
                 if i < j and link in task_var[i] and link in task_var[j]]:
        lcm = np.lcm(task_var[i][link]['T'], task_var[j][link]['T'])
        i_phi, i_t, i_l = task_var[i][link]['phi'], task_var[i][link]['T'], task_var[i][link]['L']
        j_phi, j_t, j_l = task_var[j][link]['phi'], task_var[j][link]['T'], task_var[j][link]['L']
        for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
            s.add(
                z3.Or(
                    i_phi + a * i_t >= j_phi + b * j_t + j_l,
                    j_phi + b * j_t >= i_phi + a * i_t + i_l
                )
            )

100%|██████████| 36/36 [01:02<00:00,  1.74s/it]


In [33]:
s.check()

unknown

### 2.2 IEEE 802.1 Qbv constraint

In [ ]:
for i in tqdm(task_var.keys()):
    for link in task_var[i].keys():
        s.add(0 <= task_var[i][link]['p'])
        s.add(task_var[i][link]['p'] < net_var[link]['c'])

In [ ]:
s.check()

Ideal scenario

$\begin{aligned}
&\forall\left[v_{a}, v_{b}\right] \in \mathcal{L}, \forall s_{i}^{\left[v_{a}, v_{b}\right]}, s_{j}^{\left[v_{a}, v_{b}\right]} \in \mathcal{S}, i \neq j, \\
&\forall f_{i, k}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{i}^{\left[v_{a}, v_{b}\right]}, \forall f_{j, l}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{j}^{\left[v_{a}, v_{b}\right]} \\
&\forall \alpha \in\left[0, h p_{i}^{j} / s_{i} \cdot T-1\right], \forall \beta \in\left[0, h p_{i}^{j} / s_{j} \cdot T-1\right]: \\
&\left(f_{i, k}^{\left[v_{x}, v_{a}\right]} \cdot \phi \times\left[v_{x}, v_{a}\right] . m t+\alpha \times s_{i} \cdot T+\left[v_{x}, v_{a}\right] . d+\delta \leq\right. \\
&\left.f_{j, l}^{\left[v_{y}, v_{a}\right]} \cdot \phi \times\left[v_{y}, v_{a}\right] . m t+\beta \times s_{j} . T+\left[v_{y}, v_{a}\right] . d\right) \vee \\
&\left(f_{j, l}^{\left[v_{y}, v_{a}\right]} \cdot \phi \times\left[v_{y}, v_{a}\right] . m t+\beta \times s_{j} \cdot T+\left[v_{y}, v_{a}\right] . d+\delta \leq\right. \\
&\left.f_{i, k}^{\left[v_{x}, v_{a}\right]} \cdot \phi \times\left[v_{x}, v_{a}\right] . m t+\alpha \times s_{i} \cdot T+\left[v_{x}, v_{a}\right] \cdot d\right),
\end{aligned}$

In [ ]:
# ## Idea scenario [No specific isolation]

# for i, j in tqdm([(i,j) for i in range(len(task)) for j in range(0, len(task)) if i < j]):
#     path_i = list(task_var[i].keys())
#     path_j = list(task_var[j].keys())
#     i_period, j_period = int(task.loc[i]['period']), int(task.loc[j]['period'])
    
#     for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b) 
#                         for _x, i_a_b in enumerate(path_i) 
#                         for _y, j_a_b in enumerate(path_j) 
#                         if i_a_b == j_a_b and _x>0 and _y>0 and _x != _y]:
        
#         lcm = np.lcm(task.loc[i].period, task.loc[j].period)
#         i_x_a_phi, x_a_mt, x_a_d = task_var[i][x_a]['phi'], net_var[x_a]['mt'], net_var[x_a]['d']
#         j_y_a_phi, y_a_mt, y_a_d = task_var[j][y_a]['phi'], net_var[y_a]['mt'], net_var[y_a]['d']
#         i_a_b_p = task_var[i][str(a_b)]['p']
#         j_a_b_p = task_var[j][str(a_b)]['p']
        
#         for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
#             s.add(z3.Or(
#                     i_x_a_phi * x_a_mt + a * i_period + x_a_d + NETWORK_ERROR <=
#                     j_y_a_phi * y_a_mt + b * j_period + y_a_d,
#                     j_y_a_phi * y_a_mt + b * j_period + y_a_d + NETWORK_ERROR <=
#                     i_x_a_phi * x_a_mt + a * i_period + x_a_d,
#                     i_a_b_p != j_a_b_p
#                 ))
#     print(s.check(), i, j)

Frame / Stream isolation

$\begin{aligned}
&\forall\left[v_{a}, v_{b}\right] \in \mathcal{L}, \forall s_{i}^{\left[v_{a}, v_{b}\right]}, s_{j}^{\left[v_{a}, v_{b}\right]} \in \mathcal{S}, i \neq j \\
&\forall f_{i, k}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{i}^{\left[v_{a}, v_{b}\right]}, \forall f_{j, l}^{\left[v_{a}, v_{b}\right]} \in \mathcal{F}_{j}^{\left[v_{a}, v_{b}\right]} \\
&\forall \alpha \in\left[0, h p_{i}^{j} / s_{i} \cdot T-1\right], \forall \beta \in\left[0, h p_{i}^{j} / s_{j} \cdot T-1\right]: \\
&\left(f_{j, l}^{\left[v_{a}, v_{b}\right]} \cdot \phi \times\left[v_{a}, v_{b}\right] \cdot m t+\alpha \times s_{j} . T+\delta \leq\right. \\
&\left.f_{i, k}^{\left[v_{x}, v_{a}\right]} \cdot \phi \times\left[v_{x}, v_{a}\right] . m t+\beta \times s_{i} \cdot T+\left[v_{x}, v_{a}\right] . d\right) \vee \\
&\left(f_{i, k}^{\left[v_{a}, v_{b}\right]} \cdot \phi \times\left[v_{a}, v_{b}\right] \cdot m t+\beta \times s_{i} \cdot T+\delta \leq\right. \\
&\left.f_{j, l}^{\left[v_{y}, v_{a}\right]} \cdot \phi \times\left[v_{y}, v_{a}\right] \cdot m t+\alpha \times s_{j} . T+\left[v_{y}, v_{a}\right] . d\right)
\end{aligned}$

In [ ]:
## Stream / Frame isolation

count = 0
for i, j in tqdm([(i,j) for i in range(len(task)) for j in range(0, len(task)) if i < j]):
    path_i = list(task_var[i].keys())
    path_j = list(task_var[j].keys())
    i_period, j_period = int(task.loc[i]['period']), int(task.loc[j]['period'])
    
    for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b) 
                        for _x, i_a_b in enumerate(path_i) 
                        for _y, j_a_b in enumerate(path_j) 
                        if i_a_b == j_a_b]:
        lcm = np.lcm(task.loc[i].period, task.loc[j].period)
        i_x_a_phi, j_y_a_phi, i_a_b_phi, j_a_b_phi =  task_var[i][x_a]['phi'], task_var[j][y_a]['phi'], task_var[i][a_b]['phi'], task_var[j][a_b]['phi'] 
        x_a_d, y_a_d = net_var[x_a]['d'], net_var[y_a]['d']
        i_a_b_p = task_var[i][str(a_b)]['p']
        j_a_b_p = task_var[j][str(a_b)]['p']
        
        for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
            s.add(
                z3.Or(
                    j_a_b_phi + b * j_period + sync_error <
                    i_x_a_phi + a * i_period + x_a_d,
                    i_a_b_phi + a * i_period + sync_error <
                    j_y_a_phi + b * j_period + y_a_d,
                    i_a_b_p != j_a_b_p
                )
            )

In [ ]:
s.check()

FIFO constraint - Designed by Chuanyu Jul 2 2022

In [ ]:
# ## FIFO constraint

# for i, j in tqdm([(i,j) for i in range(len(task)) for j in range(0, len(task)) if i != j]):
#     path_i = list(task_var[i].keys())
#     path_j = list(task_var[j].keys())
#     i_period, j_period = int(task.loc[i]['period']), int(task.loc[j]['period'])
    
#     for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b) 
#                         for _x, i_a_b in enumerate(path_i) 
#                         for _y, j_a_b in enumerate(path_j) 
#                         if i_a_b == j_a_b and _x>0 and _y>0]:
#         lcm = np.lcm(task.loc[i].period, task.loc[j].period)
#         i_x_a_phi, j_y_a_phi, i_a_b_phi, j_a_b_phi =  task_var[i][x_a]['phi'], task_var[j][y_a]['phi'], task_var[i][a_b]['phi'], task_var[j][a_b]['phi'] 
#         x_a_mt, y_a_mt, a_b_mt = net_var[x_a]['mt'], net_var[y_a]['mt'], net_var[a_b]['mt']
#         x_a_d, y_a_d = net_var[x_a]['d'], net_var[y_a]['d']
#         i_a_b_p = task_var[i][str(a_b)]['p']
#         j_a_b_p = task_var[j][str(a_b)]['p']
        
#         for a, b in [(a,b) for a in range(0, int(lcm / task.loc[i].period)) for b in range(0, int(lcm / task.loc[j].period))]:
#             s.add(
#                 z3.Or(
#                     i_a_b_phi * a_b_mt + a * i_period >
#                     j_a_b_phi * a_b_mt + b * j_period
#                     ,
#                     i_x_a_phi * x_a_mt + a * i_period + x_a_d <
#                     j_y_a_phi * y_a_mt + b * j_period + y_a_d
#                     ,
#                     i_a_b_p != j_a_b_p
#                 )
#             )
# print(s.check())

In [ ]:
res = s.check()
result = s.model()

In [ ]:
str(res) == 'sat'

In [ ]:
# for i in task_var:
#     print('[Task %ad] ------------------------------'%i)
#     print(i, [result[x['phi']] for x in task_var[i].values()], [result[x['p']] for x in task_var[i].values()])

## Output schedule

In [ ]:
## GCL
GCL = []
for i in task_var:
    for e in task_var[i]:
        start = result[task_var[i][e]['phi']].as_long()
        end = start + task_var[i][e]['L']
        queue = result[task_var[i][e]['p']].as_long()
        t = task_var[i][e]['T']
        for k in range(int(LCM / t)):
            GCL.append(
                [eval(e), queue, (start + k * t) * macrotick, (end  + k * t) * macrotick, LCM * macrotick]
            )

In [ ]:
## Offset
OFFSET = []
for i in task_var:
    offset = result[list(task_var[i].values())[0]['phi']].as_long()
    OFFSET.append(
        [i, 0, (list(task_var[i].values())[0]['T'] - offset) * macrotick]
    )    
# with open('RTNS16-route-CBS-16.txt', 'w') as f:
#     f.write(str(route))

In [ ]:
QUEUE = []
for i in task_var:
    for e in task_var[i]:
        QUEUE.append([i, 0, eval(e), result[task_var[i][e]['p']]])
        # queue[i][e] = result[task_var[i][e]['p']]
# with open('RTNS16-queue-CBS-16.txt', 'w') as f:
#     f.write(str(queue))

In [ ]:
ROUTE = []
for i, row in task.iterrows():
    route = eval(str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0])))))
    for h, v in enumerate(route[:-1]):
        ROUTE.append(
            [i, (v, route[h + 1])]
        )

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTNS2016-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTNS2016-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTNS2016-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTNS2016-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

In [ ]:
info = s.statistics()

In [ ]:
info.time

In [ ]:
info.max_memory